In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
Options = Options()
Options.add_argument('--headless')
driver = webdriver.Chrome(options=Options)

In [3]:
url_base = f"https://www.pagina12.com.ar"

In [4]:
soup = []
inicio_paginado = 100 #permite ir mas lejos
limite_paginado = 200 #permite ir mas lejos
for i in range(inicio_paginado,limite_paginado):
    print(i, end='\r')
    driver.get(f"{url_base}/buscar?q=milei&page={i}")
    pag =  driver.page_source
    soup.append(BeautifulSoup(pag, 'html5lib'))

In [16]:
paginas = []
fallos = []
for i in range(0,limite_paginado):
    print(i, end='\r')
    try:
        paginas.append(soup[i].find_all('div', class_='article-item__content'))
    except:
        fallos.append(i)

In [17]:
noticias = pd.DataFrame(columns=['indice','titulo','fecha','link','source','contenido','copete'])

i = -1
for pagina in paginas:
    for noticia in pagina:
        i = i + 1
        noticias.loc[i,'indice'] = i
        noticias.loc[i,'titulo'] = (noticia.get_text())
        noticias.loc[i,'link'] = f"{url_base}{noticia.find('a')['href']}"
        print(f'Procesado articulo {i}', end='\r')
        #print(f"Articulo: {i} Titulo: \n{titulo}\nLink:\n{link}\n-------------------------------------------------------------------------\n")


In [18]:
for index, row in noticias.iterrows():
    print(f"Articulo {row.indice} - {row.link}", end='\r')
    driver.get(row.link)
    pag =  driver.page_source
    row.source = BeautifulSoup(pag, 'html5lib')

In [21]:
fallos = []
for index, row in noticias.iterrows():
    #print(f"Articulo {row.indice} - {row.link}")
    try:
        contenido_source = row.source.find('div', class_='section-2-col article-main-content')
        row.contenido = contenido_source.get_text()
        copete_source = row.source.find('div', class_='section-2-col article-header')
        row.copete = copete_source.get_text()
        fecha_source = row.source.find('meta', property='article:modified_time')
        fecha = int(fecha_source.get('content'))
        fecha = datetime.datetime.fromtimestamp(fecha)
        row.fecha = fecha
    except:
        fallos.append(index) #captura los fallos por too many requests
        #print(f"Fallo en row: {index}")

In [22]:
#corremos de nuevo las lineas donde falló
for f in fallos:
    try:
        row = noticias.loc[f]
        print(f"Articulo {row.indice} - {row.link}", end='\r')
        driver.get(row.link)
        pag =  driver.page_source
        row.source = BeautifulSoup(pag, 'html5lib')
    except:
        print(f"done")

#luego hay que volver a ejecutar el codigo de limpieza

In [23]:
noticias.drop('source', axis = 1, inplace=True)
noticias.to_excel('02_pagina12_parte2.xlsx')

In [24]:
driver.close()